# Bayesian Naive Bayes

### Spring 2022 - Course Project

### Authors: Marc Taberner, Otis Carpay

In this notebook we will first study the standard frequentist approach to the Naive Bayes Classifier. To do so, we define a base class for the classifier and subclass it for the Bernoulli, Multinomial, and Gaussian models. We test these classifiers on two text classification datasets and two continuous numerical datasets.

The second part of the notebook treats a Bayesian approach. We will compare the frequentist approach with analytical Bayesian implementations of the Bernoulli and the Multinomial event model, and Stan implementation of all models.

## Frequentist approach

In [259]:
import pandas as pd
import numpy as np
from scipy.stats import multinomial, norm, bernoulli
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

### Class for Naive Bayes Classifier:

Here we define the base class, which will be subclassed for the specific event models. These event models will implement a `compute_params` function to compute the MLE estimators for the parameters (sufficient statistics) of the distributions, and a `compute_probabilities` function to compute the probabilities $P(X_{j} = x_{j}|C = i)$ for a given subset of the data of class $i$.

The `fit` function calls the `compute_parameters` for the subset of data for every class and computes the class_probabilites $P(C = i)$.
The function `posteriors` (following the term classically used for Naive Bayes and not referring to a Bayesian concept in this case) calculates
$$g_{i}(x) = \prod_{j= 1}^{n}P(X_{j} = x_{j}|C = i) P(C = i)$$
 and `predict` calculates 
$$\argmax g_{i}(x).$$

In [366]:
class NaiveBayes:
  def compute_params(self, X):
    # to be defined in a subclass
    # function to compute the distribution parameters
    # should return an list of parameter arrays of shape (classes, features)
    return [[]]

  def compute_probabilities(self, X, params):
    # to be defined in a subclass
    # function to compute the probabilities from the parameters
    pass

  def fit(self, X, y):
    # retrieve the number of samples and features from X
    self.n_samples, self.n_features = X.shape

    # collect the individual classes, and record their counts to compute priors
    self.classes, counts = np.unique(y, return_counts=True)

    # compute the parameters for every class - we transpose the array so that
    # the final dimensions are (parameters, classes, features)
    self.params = np.array(
      [self.compute_params(X[c==y]) for c in self.classes]
    ).transpose(1, 0, 2)

    # compute the class probabilities from the counts
    self.class_probs = np.log(counts/self.n_samples)

    return self

  def posteriors(self, X):
    # reshape X to fit the array dimensions (samples, classes, features)
    X = np.reshape(X, (-1, 1, self.n_features))

    # compute the probabilities P(X_j = x_j | C = i)
    probs = self.compute_probabilities(X, *self.params[:,np.newaxis])

    # finally compute the posteriors by summing all log probs
    return probs.sum(axis=2) + self.class_probs[np.newaxis]

  def predict(self, X):
    # predict the class with the maximum probability
    return self.classes[np.argmax(self.posteriors(X), axis=1)]

#### Bernoulli model

First we define the Bernoulli model. We include a smoothing parameters $\alpha$ and set it to a small $10^{-10}$ by default to avoid computational problems. We clip the data to a maximum of 1 so we can apply the classifier to a multinomial dataset without any extra work.

In [124]:
class BernoulliNB(NaiveBayes):
  def __init__(self, alpha=1e-10):
    self.alpha = alpha

  def compute_params(self, X):
    # the MLE estimations are easily calculated by taking the mean
    return [X.clip(max=1).mean(axis=0) + self.alpha]
  
  def compute_probabilities(self, X, phi):
    return bernoulli.logpmf(X.clip(max=1), phi)

#### Multinomial model

Like the Bernoulli model, we include a smoothing parameter

In [125]:
class MultinomialNB(NaiveBayes):
  def __init__(self, alpha=1e-10):
    self.alpha = alpha

  def compute_params(self, X):
    counts = X.sum(axis=0) + self.alpha
    return [counts/counts.sum()]

  def compute_probabilities(self, X, phi):
    # we need to add a new axis to conform to the shape expected by
    # the posteriors function: the feature axis is of size 1 since
    # the distribution gives the probability of a whole sample
    return multinomial.logpmf(X, X.sum(axis=2), phi)[...,np.newaxis]

### Gaussian model

Finally, the Gaussian model perhaps has the most straightforward code

In [126]:
class GaussianNB(NaiveBayes):
  def compute_params(self, X):
    return [X.mean(axis=0), X.std(axis=0)]

  def compute_probabilities(self, X, mean, sd):
    return norm.logpdf(X, mean, sd)

## Experiments

In [ ]:
def evaluate_clf(clf, xtrain, xtest, ytrain, ytest):
  clf.fit(xtrain, ytrain)
  y_pred_test = clf.predict(xtest)
  y_pred_train = clf.predict(xtrain)
  print(f"The train error is: {(y_pred_train == ytrain).mean():.3f}")
  print(f"The test error is: {(y_pred_test == ytest).mean():.3f}")

### Spam classification

Given an e-mail message, determine wheather such message is Spam or Ham, text classification task:

In [251]:
data = pd.read_csv("data/spam.csv", encoding='latin-1')
data = data[["class", "message"]]
data

,class,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


We transform the documents to multinomial points with the sklearn `CountVectorizer` class. It transforms the data to vectors with the number of occurrences for each word. We set it to excludes words from a list of common words called 'stop words', and limit the words to the 1000 top words for computational speed. In our tests, the effect on accuracy was negligible.

In [252]:
x = np.array(data["message"])
y = np.array(data["class"])

cv = CountVectorizer(stop_words='english', max_features=1000)

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42, shuffle = True)

xtrain = cv.fit_transform(xtrain).toarray()
xtest = cv.transform(xtest).toarray()

Note that we have the value counts with valus up to 18 occurrences, we can work then with Bernoulli, Multinomial and Gaussian Naive Bayes, althought it is clear that most likely multinomial will be the most efficient:

In [253]:
np.unique(xtrain)

array([ 0,  1,  2,  3,  4,  5,  6,  8, 10, 15, 18])

#### Using Bernoulli Naive Bayes

In [254]:
evaluate_clf(BernoulliNB(alpha=1e-4), xtrain, xtest, ytrain, ytest)

The train error is: 0.990
The test error is: 0.984


#### Using Multinomial Naive Bayes:

In [255]:
evaluate_clf(MultinomialNB(alpha=2), xtrain, xtest, ytrain, ytest)

The train error is: 0.985
The test error is: 0.983


### Using Gaussian Naive Bayes

In [256]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(xtrain) #to standarize features to a zero mean and 1 variance!
xtrain = scaler.transform(xtrain)
xtest = scaler.transform(xtest)

evaluate_clf(GaussianNB(), xtrain, xtest, ytrain, ytest)

/Users/otis/Documents/uni/2/Bayes/Project/venv/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:1945: RuntimeWarning: divide by zero encountered in divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/otis/Documents/uni/2/Bayes/Project/venv/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:1945: RuntimeWarning: invalid value encountered in divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)


The train error is: 0.134
The test error is: 0.135


In [257]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(xtrain) #to standarize features to a zero mean and 1 variance!
xtrain = scaler.transform(xtrain)
xtest = scaler.transform(xtest)

evaluate_clf(GaussianNB(), xtrain, xtest, ytrain, ytest)

/Users/otis/Documents/uni/2/Bayes/Project/venv/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:1945: RuntimeWarning: divide by zero encountered in divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/otis/Documents/uni/2/Bayes/Project/venv/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:1945: RuntimeWarning: invalid value encountered in divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)


The train error is: 0.866
The test error is: 0.865


### BBC Article Classification

This dataset is a collection of BBC articles annotated with the type of the article

In [241]:
data = pd.read_csv("data/articles.csv")
data

,news,type
0,China had role in Yukos split-up\n \n China le...,business
1,Oil rebounds from weather effect\n \n Oil pric...,business
2,Indonesia 'declines debt freeze'\n \n Indonesi...,business
3,$1m payoff for former Shell boss\n \n Shell is...,business
4,US bank in $515m SEC settlement\n \n Five Bank...,business
...,...,...
2220,Microsoft launches its own search\n \n Microso...,tech
2221,Warnings about junk mail deluge\n \n The amoun...,tech
2222,Microsoft gets the blogging bug\n \n Software ...,tech
2223,Gamers snap up new Sony PSP\n \n Gamers have b...,tech


We have the following five article types:

In [242]:
data['type'].unique()

array(['business', 'entertainment', 'politics', 'sport', 'tech'],
      dtype=object)

We transform the dataset in the same way as the previous dataset

In [243]:
x = np.array(data["news"])
y = np.array(data["type"])

cv = CountVectorizer(stop_words='english', max_features=1000) # count the amout of occurrences for each word
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42, shuffle = True)
xtrain = cv.fit_transform(xtrain).toarray()
xtest = cv.transform(xtest).toarray()

### Using Bernoulli Naive Bayes:

In [244]:
evaluate_clf(BernoulliNB(alpha=1e-7), xtrain, xtest, ytrain, ytest)

The train error is: 0.976
The test error is: 0.937


### Using Multinomial Naive Bayes:

In [245]:
evaluate_clf(MultinomialNB(alpha=2), xtrain, xtest, ytrain, ytest)

The train error is: 0.975
The test error is: 0.951


### Using Gaussian Naive Bayes

In [246]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(xtrain) #to standarize features to a zero mean and 1 variance!
xtrain = scaler.transform(xtrain)
xtest = scaler.transform(xtest)

evaluate_clf(GaussianNB(), xtrain, xtest, ytrain, ytest)

/Users/otis/Documents/uni/2/Bayes/Project/venv/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:1945: RuntimeWarning: divide by zero encountered in divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/otis/Documents/uni/2/Bayes/Project/venv/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:1945: RuntimeWarning: invalid value encountered in divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)


The train error is: 0.222
The test error is: 0.258


## Raisin dataset

The raisin dataset was from a study of machine vision systems in order to distinguish between two distinct varieties of raisins, Kecimen and Besni, a total of 900 pices of raisin were obtained and distinct metric values were recorded and included in the dataset. 

In [200]:
df = pd.read_csv('data/Raisin_Dataset.csv')
x = np.array(df.drop("Class", axis=1))
y = np.array(df["Class"])
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42, shuffle = True)

In [201]:
df

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter,Class
0,87524,442.246011,253.291155,0.819738,90546,0.758651,1184.040,Kecimen
1,75166,406.690687,243.032436,0.801805,78789,0.684130,1121.786,Kecimen
2,90856,442.267048,266.328318,0.798354,93717,0.637613,1208.575,Kecimen
3,45928,286.540559,208.760042,0.684989,47336,0.699599,844.162,Kecimen
4,79408,352.190770,290.827533,0.564011,81463,0.792772,1073.251,Kecimen
...,...,...,...,...,...,...,...,...
895,83248,430.077308,247.838695,0.817263,85839,0.668793,1129.072,Besni
896,87350,440.735698,259.293149,0.808629,90899,0.636476,1214.252,Besni
897,99657,431.706981,298.837323,0.721684,106264,0.741099,1292.828,Besni
898,93523,476.344094,254.176054,0.845739,97653,0.658798,1258.548,Besni


Naturally given that the data presented is continous, of the studied models we can only apply the Gaussian Naive Bayes. Let's observe the yielding results:

In [202]:
scaler = StandardScaler().fit(xtrain) #to standarize features to a zero mean and 1 variance!
xtrain = scaler.transform(xtrain)
xtest = scaler.transform(xtest)

evaluate_clf(GaussianNB(), xtrain, xtest, ytrain, ytest)

The train error is: 0.838
The test error is: 0.839


Note that again, as in the Spam or Ham dataset, we have data with multiplicites, we expect the Multinomial Naive Bayes to perform the best, but we can actually perform tests in the 3 distinct models presented.

## Iris Dataset

This dataset is similar to the Raisin dataset, this set of data containd 50 distinct samples of 3 distinct Iris plants, Iris setosa, Iris viginica and Iris Versicolor. 4  distinct features were recorded for each plant, we would want to be able to tell apart each of this type of Iris just with this given 4 features. 

In [206]:
df = pd.read_csv('data/Iris.csv')
df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


As in the Raisin Dataset, data is continous, and hence, the only course of action would be to approach it using a Gaussian Naive Bayes.

In [213]:
x = np.array(df.drop("Species", axis=1))
y = np.array(df["Species"])
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42, shuffle = True)

In [214]:
scaler = StandardScaler().fit(xtrain) #to standarize features to a zero mean and 1 variance!
xtrain = scaler.transform(xtrain)
xtest = scaler.transform(xtest)

evaluate_clf(GaussianNB(), xtrain, xtest, ytrain, ytest)

The train error is: 0.992
The test error is: 1.000


## Bayesian approach

In this part we will take a Bayesian approach to the Naive Bayes classifier. That is, we define prior distributions over the parameters of the classifier. We define first analytical formulations of the Bayesian version of the Bernoulli and Multinomial models. Then we obtain the parameters from the data via probabilistic inference with Stan.

### Analytical

In [288]:
from scipy.stats import beta, dirichlet

#### Bernoulli

In [388]:
class BayesBernoulliNB(BernoulliNB):
  def __init__(self, alpha=1, beta=1):
    # set the parameters for the prior beta distribution
    self.a = alpha,
    self.b = beta

  def compute_params(self, X):
    X = X.clip(max=1)
    return [beta.mean(self.a + X.sum(axis=0), self.b + len(X) - X.sum(axis=0))]

#### Multinomial

In [263]:
class BayesMultinomialNB(MultinomialNB):
  def __init__(self, alpha=[1]):
    # set the parameters for the prior beta distribution
    self.a = np.array(alpha)

  def compute_params(self, X):
    return [dirichlet.mean(self.a + X.sum(axis=0))]

## Stan Implementation

In [367]:
class StanNB(NaiveBayes):
  def __init__(self, model_path, random_state=None):
    self.seed = random_state
    with open(model_path) as f:
      self.stan_model = f.read()

  def fit(self, X, y, data, param_names):
    self.stan_data = data
    self.param_names = param_names

    posterior = stan.build(self.stan_model, data=self.stan_data, random_seed=self.seed)
    self.stan_fit = posterior.sample(num_chains=4, num_samples=1000)

    super().fit(X, y)
    
    self.class_probs = self.stan_fit['theta'].mean(axis=-1)
    self.params = np.array(
      [self.stan_fit[param].mean(axis=-1) for param in param_names]
    )

    return self

#### Bernoulli

In [368]:
class StanBernoulliNB(StanNB):
  def __init__(self, *, class_prior, param_prior, random_state=None):
    self.alpha = class_prior
    self.beta = param_prior
    super().__init__(model_path='model_bernoulli.stan', random_state=random_state)

  def compute_probabilities(self, X, phi):
    return BernoulliNB.compute_probabilities(self, X, phi)
  
  def fit(self, X, y):
    classes, y_int = np.unique(y, return_inverse=True)

    data = {
      'C': len(classes),    # num categories
      'K': X.shape[1],      # num features
      'N': X.shape[0],      # num samples
      'y': y_int + 1,       # category of sample n
      'x': X.clip(max=1),   # sample n
      'alpha': self.alpha,  # class prior
      'beta': self.beta     # feature prior
    }

    return super().fit(X, y, data, ['phi'])

#### Multinomial

In [369]:
class StanMultinomialNB(StanNB):
  def __init__(self, *, class_prior, param_prior, random_state=None):
    self.alpha = class_prior
    self.beta = param_prior
    super().__init__(model_path='model_multinomial.stan', random_state=random_state)

  def compute_probabilities(self, X, phi):
    return MultinomialNB.compute_probabilities(self, X, phi)
  
  def fit(self, X, y):
    classes, y_int = np.unique(y, return_inverse=True)

    idx = X.nonzero()
    values = X[idx]
    word_ids = np.repeat(idx[1],values)
    doc_ids = np.repeat(idx[0],values)

    data = {
      'K': len(classes),    # num classes
      'V': X.shape[1],      # num words
      'M': X.shape[0],      # num docs
      'N': len(word_ids),   # total word instances
      'z': y_int + 1,       # class for doc m
      'w': word_ids + 1,    # word n
      'doc': doc_ids + 1,   # doc ID for word n
      'alpha': self.alpha,  # class prior
      'beta': self.beta     # feature prior
    }

    return super().fit(X, y, data, ['phi'])

#### Gaussian

In [370]:
class StanGaussianNB(StanNB):
  def __init__(self, *, class_prior, mu_prior, sigma_prior, random_state=None):
    self.alpha = class_prior
    self.mu_mu, self.mu_sigma = mu_prior,
    self.sigma_alpha, self.sigma_beta = sigma_prior
    super().__init__(model_path='model_gaussian.stan', random_state=random_state)

  def compute_probabilities(self, X, mean, sd):
    return GaussianNB.compute_probabilities(self, X, mean, sd)
  
  def fit(self, X, y):
    classes, y_int = np.unique(y, return_inverse=True)

    data = {
      'C': len(classes),                # num classes
      'K': X.shape[1],                  # num features
      'N': X.shape[0],                  # num samples
      'y': y_int + 1,                   # class of sample n
      'x': X,                           # sample n
      'alpha': self.alpha,              # class prior
      'mu_mu': self.mu_mu,              # feature mean prior
      'mu_sigma': self.mu_sigma,        # feature mean prior
      'sigma_alpha': self.sigma_alpha,  # feature sd prior
      'sigma_beta': self.sigma_beta     # feature sd prior
    }

    return super().fit(X, y, data, ['mu', 'sigma'])

## Experiments

### Spam dataset

In [353]:
data = pd.read_csv("data/spam.csv", encoding='latin-1')
x = np.array(data["message"])
y = np.array(data["class"])

cv = CountVectorizer(stop_words='english', max_features=100)

xtrain, xtest, ytrain, ytest = train_test_split(x, y, train_size=100, random_state=42, shuffle = True)

xtrain = cv.fit_transform(xtrain).toarray()
xtest = cv.transform(xtest).toarray()

In [404]:
evaluate_clf(BernoulliNB(alpha=0.1), xtrain, xtest, ytrain, ytest)

The train error is: 0.930
The test error is: 0.878


In [410]:
evaluate_clf(BayesBernoulliNB(alpha=0.1), xtrain, xtest, ytrain, ytest)

The train error is: 0.980
The test error is: 0.910


In [413]:
evaluate_clf(
  StanBernoulliNB(
    class_prior=[0.1,0.1],
    param_prior=np.full((xtrain.shape[1], 2), 0.1),
    random_state=42),
  xtrain, xtest, ytrain, ytest)

Building...



Building: found in cache, done.Messages from stanc:
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Sampling:   1% (103/8000)
Sampling:   3% (202/8000)
Sampling:   4% (301/8000)
Sampling:   5% (400/8000)
Sampling:   6% (500/8000)
Sampling:   8% (600/8000)
Sampling:   9% (700/8000)
Sampling:  10% (800/8000)
Sampling:  11% (900/8000)
Sampling:  12% (1000/8000)
Sampling:  14% (1100/8000)
Sampling:  15% (1200/8000)
Sampling:  16% (1300/8000)
Sampling:  18% (1400/8000)
Sampling:  19% (1500/8000)
Sampling:  20% (1600/8000)
Sampling:  21% (1700/8000)
Sampling:  22% (1800/8000)
Sampling:  24% (1900/8000)
Sampling:  25% (2000/8000)
Sampling:  26% (2100/8000)
Sampling:  28% (2200/8000)
Sampling:  29% (2300/8000)
Sampling:  30% (2400/8000)
Sampling:  31% (2500/8000)
Sampling:  32% (2600/8000)
Sampling:  34% (2700/8000)
Sampling:  35% (2800/8000)
Sampling:  36% (2900/8000)
Sampling:  38% (3000/8000)
Sampling:  39% (3100/8000)
Sampling

The train error is: 0.970
The test error is: 0.906


In [399]:
evaluate_clf(MultinomialNB(alpha=0.1), xtrain, xtest, ytrain, ytest)

The train error is: 0.980
The test error is: 0.912


In [398]:
evaluate_clf(BayesMultinomialNB(alpha=[0.1]), xtrain, xtest, ytrain, ytest)

The train error is: 0.980
The test error is: 0.912


In [415]:
evaluate_clf(
  StanMultinomialNB(
    class_prior=[0.1, 0.1],
    param_prior=np.full(xtrain.shape[1], 0.1),
    random_state=42),
  xtrain, xtest, ytrain, ytest)

Building...



Building: found in cache, done.Messages from stanc:
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Sampling:   1% (103/8000)
Sampling:   3% (202/8000)
Sampling:   4% (301/8000)
Sampling:   5% (400/8000)
Sampling:   6% (500/8000)
Sampling:   8% (600/8000)
Sampling:   9% (700/8000)
Sampling:  10% (800/8000)
Sampling:  11% (900/8000)
Sampling:  12% (1000/8000)
Sampling:  14% (1100/8000)
Sampling:  15% (1200/8000)
Sampling:  16% (1300/8000)
Sampling:  18% (1400/8000)
Sampling:  19% (1500/8000)
Sampling:  20% (1600/8000)
Sampling:  21% (1700/8000)
Sampling:  22% (1800/8000)
Sampling:  24% (1900/8000)
Sampling:  25% (2000/8000)
Sampling:  26% (2100/8000)
Sampling:  28% (2200/8000)
Sampling:  29% (2300/8000)
Sampling:  30% (2400/8000)
Sampling:  31% (2500/8000)
Sampling:  32% (2600/8000)
Sampling:  34% (2700/8000)
Sampling:  35% (2800/8000)
Sampling:  36% (2900/8000)
Sampling:  38% (3000/8000)
Sampling:  39% (3100/8000)
Sampling

The train error is: 0.950
The test error is: 0.873


### Article dataset

In [416]:
data = pd.read_csv("data/articles.csv")
x = np.array(data["news"])
y = np.array(data["type"])

cv = CountVectorizer(stop_words='english', max_features=100)

xtrain, xtest, ytrain, ytest = train_test_split(x, y, train_size=100, random_state=42, shuffle=True)

xtrain = cv.fit_transform(xtrain).toarray()
xtest = cv.transform(xtest).toarray()

In [448]:
num_classes = len(np.unique(y))

In [445]:
evaluate_clf(BernoulliNB(alpha=0.01), xtrain, xtest, ytrain, ytest)

The train error is: 0.940
The test error is: 0.816


In [452]:
evaluate_clf(BayesBernoulliNB(alpha=0.1), xtrain, xtest, ytrain, ytest)

The train error is: 0.960
The test error is: 0.818


In [454]:
evaluate_clf(
  StanBernoulliNB(
    class_prior=np.full(num_classes, 0.1),
    param_prior=np.full((xtrain.shape[1], 2), 0.1),
    random_state=42),
  xtrain, xtest, ytrain, ytest)

Building...



Building: found in cache, done.Messages from stanc:
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Sampling:   1% (103/8000)
Sampling:   3% (202/8000)
Sampling:   4% (301/8000)
Sampling:   5% (400/8000)
Sampling:   6% (500/8000)
Sampling:   8% (600/8000)
Sampling:   9% (700/8000)
Sampling:  10% (800/8000)
Sampling:  11% (900/8000)
Sampling:  12% (1000/8000)
Sampling:  14% (1100/8000)
Sampling:  15% (1200/8000)

In [ ]:
evaluate_clf(MultinomialNB(alpha=0.1), xtrain, xtest, ytrain, ytest)

The train error is: 0.980
The test error is: 0.912


In [ ]:
evaluate_clf(BayesMultinomialNB(alpha=[0.1]), xtrain, xtest, ytrain, ytest)

The train error is: 0.980
The test error is: 0.912


In [ ]:
evaluate_clf(
  StanMultinomialNB(
    class_prior=[1,1],
    param_prior=np.full(xtrain.shape[1], 1),
    random_state=42),
  xtrain, xtest, ytrain, ytest)

Building...



Building: found in cache, done.Messages from stanc:
Sampling:   0%
Sampling: 100%, done.
Messages received during sampling:
  Gradient evaluation took 0.003971 seconds
  1000 transitions using 10 leapfrog steps per transition would take 39.71 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 0.001222 seconds
  1000 transitions using 10 leapfrog steps per transition would take 12.22 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 0.000341 seconds
  1000 transitions using 10 leapfrog steps per transition would take 3.41 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 0.001379 seconds
  1000 transitions using 10 leapfrog steps per transition would take 13.79 seconds.
  Adjust your expectations accordingly!


The train error is: 0.930
The test error is: 0.873
